## Importing the data 

In [8]:
import pandas as pd

In [2]:
users = pd.read_csv("/Users/friedatims-jansma/Desktop/coding_challenge_csv/users.csv")

In [3]:
users

,user_id,first_name,last_name,house
0,1,Harry,Potter,Gryffindor
1,2,Ron,Wesley,Gryffindor
2,3,Hermonie,Granger,Gryffindor
3,4,Ginny,Weasley,Gryffindor
4,5,Draco,Malfoy,Slytherin
5,6,Tom,Riddle,Slytherin
6,7,Luna,Lovegood,Ravenclaw
7,8,Cho,Chang,Ravenclaw
8,9,Cedric,Diggory,Hufflepuff


In [4]:
follows = pd.read_csv("/Users/friedatims-jansma/Desktop/coding_challenge_csv/follows.csv")

In [6]:
follows.head()

,user_id,follows,date
0,1,2,1993-09-01
1,2,1,1989-01-01
2,3,1,1993-07-01
3,2,3,1994-10-10
4,3,2,1995-03-01


### How many users are there in each house?

In [5]:
import sqlite3
conn = sqlite3.connect('/Users/friedatims-jansma/code/FriedaTims-Jansma/Le-Wagon-Career-Week-Challenge/notebooks/SQL_challenge.db')
c = conn.cursor()



In [12]:
c.execute("SELECT COUNT(users.user_id), users.house FROM users GROUP BY users.house") 
question_1 = c.fetchall()
pd.DataFrame(question_1, columns=['#_Of_Users', 'House'])

,#_Of_Users,House
0,4,Gryffindor
1,1,Hufflepuff
2,2,Ravenclaw
3,2,Slytherin


### List all following links that were created before September 1st, 1993